![imagenes](logo.png)

# SD vs MADe

Ya hemos aprendido algunas medidas robustas: media vs mediana; rango vs rango intercuartílico; y desviación típica vs MAD.

Ahora vamos a enfocarnos en cómo detectar datos atípico.

## SD

La mayoría de los métodos de detección de atípicos consisten en hallar intervalos fuera de los cuales considerar cualquier dato como atípico. 

El método **SD** es un método clásico de detección, debido a que es muy sencillo de usar. Funciona bien con distribuciones simétricas con forma de campana, como las gaussianas.

Se  basa en la regla de los 3-sigmas que estudiamos previamente: en una distribución gaussiana, aproximadamente el 68% de los datos esté en $(\mu-\sigma,\mu+\sigma)$; aproximadamente el 95% de los datos esté en $(\mu-2\sigma,\mu+2\sigma)$; y aproximadamente el 99.7% de los datos esté en $(\mu-3\sigma,\mu+3\sigma)$. Con esto en mente, las observaciones que están a 2 o 3 desviaciones de la media, se consideran muy alejadas y se puede sospechar que son valores atípicos.

Se dice que un atípico es de tipo 2SD si cae fuera del intervalo $(\mu-2\sigma,\mu+2\sigma)$; y que es de tipo 3D si cae fuera del intervalo $(\mu-3\sigma,\mu+3\sigma)$. Los atípicos 3SD siempre son también del tipo 2SD. 

No obstante, **la presencia de atípicos muy extremos puede enmascarar atípicos no tan extremos**, que es lo que se conoce como *masking effect* (efecto de máscara). 

Un atípico *enmascara* a otro, si el segundo puede ser detectado solo sin la presencia del primero. Lo que sucede es que la media y la desviación se ven afectadas por los atípicos muy extremos, y la media se desplaza tanto hacia él que luego la distancia de los enmascarados hacia la media no es tan grande.

Por otra parte, el fenómeno inverso también puede suceder: que exista un atípico o grupo de atípicos que hace que alguna observación también sea atípica. Es decir, que la desaparición de los primeros implica que este último deje de ser atípico.  Esto es lo que se conoce como **swamping effect**.

**Ejemplo.** Consideremos las tablas datos_gauss.csv y datos_gauss_sin.csv de nuestro repositorio de datos.
```r
################################################################
########################      EN R       #######################
################################################################

datos_gauss = read.csv("datos_gauss.csv")
datos_gauss_sin = read.csv("datos_gauss_sin.csv")

datos = datos_gauss$valor
datos_sin = datos_gauss_sin$valor

####################### Datos con atípicos extremos

media_datos = mean(datos,na.rm=TRUE)
sd_datos = sd(datos,na.rm=TRUE)

###########  Atípicos 2D de datos con atípicos extremos

lim_2inf_datos = media_datos - 2*sd_datos
lim_2sup_datos = media_datos + 2*sd_datos

datos[(datos < lim_2inf_datos) | (datos > lim_2sup_datos)]

###########  Atípicos 3D de datos con atípicos extremos

lim_3inf_datos = media_datos - 3*sd_datos
lim_3sup_datos = media_datos + 3*sd_datos

datos[(datos < lim_3inf_datos) | (datos > lim_3sup_datos)]

#############################
#############################
#############################

####################### Datos sin atípicos extremos

media_datos_sin = mean(datos_sin,na.rm=TRUE)
sd_datos_sin = sd(datos_sin,na.rm=TRUE)

###########  Atípicos 2D de datos sin atípicos extremos

lim_2inf_datos_sin = media_datos_sin - 2*sd_datos_sin
lim_2sup_datos_sin = media_datos_sin + 2*sd_datos_sin

datos_sin[(datos_sin < lim_2inf_datos_sin) | (datos_sin > lim_2sup_datos_sin)]

###########  Atípicos 3D de datos sin atípicos extremos

lim_3inf_datos_sin = media_datos_sin - 3*sd_datos_sin
lim_3sup_datos_sin = media_datos_sin + 3*sd_datos_sin

datos_sin[(datos_sin < lim_3inf_datos_sin) | (datos_sin > lim_3sup_datos_sin)]

################################################################
```

```python

################################################################
########################     EN PYTHON    ######################
################################################################

import pandas as pd
import numpy as np

datos_gauss = pd.read_csv("datos_gauss.csv")
datos_gauss_sin = pd.read_csv("datos_gauss_sin.csv")

datos = datos_gauss["valor"]
datos_sin = datos_gauss_sin["valor"]

####################### Datos con atípicos extremos

media_datos = np.mean(datos)
sd_datos = np.std(datos, ddof=1)

###########  Atípicos 2D de datos con atípicos extremos

lim_2inf_datos = media_datos - 2 * sd_datos
lim_2sup_datos = media_datos + 2 * sd_datos

datos[(datos < lim_2inf_datos) | (datos > lim_2sup_datos)]

###########  Atípicos 3D de datos con atípicos extremos

lim_3inf_datos = media_datos - 3 * sd_datos
lim_3sup_datos = media_datos + 3 * sd_datos

datos[(datos < lim_3inf_datos) | (datos > lim_3sup_datos)]

#############################
#############################
#############################

####################### Datos sin atípicos extremos

media_datos_sin = np.mean(datos_sin)
sd_datos_sin = np.std(datos_sin, ddof=1)

###########  Atípicos 2D de datos sin atípicos extremos

lim_2inf_datos_sin = media_datos_sin - 2 * sd_datos_sin
lim_2sup_datos_sin = media_datos_sin + 2 * sd_datos_sin

datos_sin[(datos_sin < lim_2inf_datos_sin) | (datos_sin > lim_2sup_datos_sin)]

###########  Atípicos 3D de datos sin atípicos extremos

lim_3inf_datos_sin = media_datos_sin - 3 * sd_datos_sin
lim_3sup_datos_sin = media_datos_sin + 3 * sd_datos_sin

datos_sin[(datos_sin < lim_3inf_datos_sin) | (datos_sin > lim_3sup_datos_sin)]

################################################################
```

Finalmente, debemos comentar que las ventajas que tiene el SD es que es un método muy sencillo de implementar y funciona muy bien bajo gaussianidad. Pero si la distribución es asimétrica, no es un buen método de detección de atípicos. Además está basado en estimadores que no son robustos: la media y la desviación; sin mencionar que es sensible a outliers muy extremos dando como resultados posibles problemas de enmascaramiento.

## MADe

Ahora conviene preguntarnos cómo actuar en caso de no gaussianidad. De forma similar el método SD, el método MADe utilizará la mediana y el MAD. Por lo tanto se trata de un método más robusto. De esta manera, se define el método MADe como 

* Atípico de clase 2: Si queda fuera del intervalo $(Mediana-2MAD,Mediana+2MAD)$.

* Atípico de clase 3: Si queda fuera del intervalo $(Mediana-3MAD,Mediana+3MAD)$.

Las ventajas que ofrece este método son que es un método sencillo de implementar; utiliza estimadores robustos y funciona bien aunque haya asimetría.

**Ejemplo.** Consideremos las edades de nuestra tabla alumnos_scidata.csv

<img src="im021.png" style="display:block; margin:auto;">

Estos datos no siguen una distribución simétrica. Intentemos capturar los atípicos de tipo 2 y 3 con el método SD y con el método MADe

```r
################################################################
########################      EN R       #######################
################################################################

alumnos = read.csv("alumnos_scidata.csv")
edades = alumnos$Edad
edades = edades[!is.na(edades)]

media = mean(edades,na.rm=TRUE)
desv = sd(edades,na.rm=TRUE)
mediana = median(edades,na.rm=TRUE)
MAD = mad(edades,na.rm=TRUE,constant=1)

###########  Atípicos 2 con SD

lim_2inf_SD = media - 2*desv
lim_2sup_SD = media + 2*desv

edades[(edades < lim_2inf_SD) | (edades > lim_2sup_SD)]

###########  Atípicos 3 con SD

lim_3inf_SD = media - 3*desv
lim_3sup_SD = media + 3*desv

edades[(edades < lim_3inf_SD) | (edades > lim_3sup_SD)]

###########  Atípicos 2 con MAD

lim_2inf_MADe = mediana - 2*MAD
lim_2sup_MADe = mediana + 2*MAD

edades[(edades < lim_2inf_MADe) | (edades > lim_2sup_MADe)]

###########  Atípicos 3 con MAD

lim_3inf_MADe = mediana - 3*MAD
lim_3sup_MADe = mediana + 3*MAD

edades[(edades < lim_3inf_MADe) | (edades > lim_3sup_MADe)]
```

```python
################################################################
########################    EN PYTHON   ########################
################################################################

import numpy as np
import pandas as pd
import scipy.stats as stats

alumnos = pd.read_csv("alumnos_scidata.csv")
edades = alumnos["Edad"]
edades = edades[~np.isnan(edades)]

media = np.mean(edades)
desv = np.std(edades, ddof=1)
mediana = np.median(edades)
MAD = stats.median_abs_deviation(edades)

###########  Atípicos 2 con SD

lim_2inf_SD = media - 2 * desv
lim_2sup_SD = media + 2 * desv

edades[(edades < lim_2inf_SD) | (edades > lim_2sup_SD)]

###########  Atípicos 3 con SD

lim_3inf_SD = media - 3 * desv
lim_3sup_SD = media + 3 * desv

edades[(edades < lim_3inf_SD) | (edades > lim_3sup_SD)]

###########  Atípicos 2 con MAD

lim_2inf_MADe = mediana - 2 * MAD
lim_2sup_MADe = mediana + 2 * MAD

edades[(edades < lim_2inf_MADe) | (edades > lim_2sup_MADe)]

###########  Atípicos 3 con MAD

lim_3inf_MADe = mediana - 3 * MAD
lim_3sup_MADe = mediana + 3 * MAD

edades[(edades < lim_3inf_MADe) | (edades > lim_3sup_MADe)]
```

Finalmente, es interesante notar que **el método MADe también nos proporciona un estimador robusto para la desviación en el caso de gaussianas:**

$$\mbox{ si }MADe=1.483*MAD\mbox{ entonces }\hat{\sigma}=MADe$$